# Analizando la página objetivo

Este notebook tiene como objetivo analizar la página a la cual queremos hacer webscrapping:

https://www.isciii.es/QueHacemos/Servicios/VigilanciaSaludPublicaRENAVE/EnfermedadesTransmisibles/Paginas/InformesCOVID-19.aspx

Esta página contiene los datos actualizados por parte del gobierno de los casos de contagio por coronavirus. El obejtivo de este notebook es en primer lugar, realziar webscrapping para descargar los PDFs que redirige. Seguidamente, para recolectar los datos del pdf realizaremos pdf scraping. Todos los pasos se automatizan también en scripts en el proyecto.

In [46]:
# libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Tag

In [47]:
# Descargamos la página objetivo
page = requests.get(
    'https://www.isciii.es/QueHacemos/Servicios/VigilanciaSaludPublicaRENAVE/'
    'EnfermedadesTransmisibles/Paginas/InformesCOVID-19.aspx'
)
if page.status_code == 200:
    print('Página descargada con éxito')

Página descargada con éxito


In [48]:
# creamos la sopa a traves de la página
soup = BeautifulSoup(page.content)

Las páginas objetivo se encuentran dentro de un a href.

In [49]:
# buscamos bajo el id donde están los links de interés
id_links = soup.find(id='ctl00_PlaceHolderMain_ctl02__ControlWrapper_RichHtmlField')

# para cada hijo buscamos a y guardamos su enlace
links_pdfs = []
for child in id_links.children:
    a_tag = child.find('a')
    if isinstance(a_tag, Tag):
        # print(a_tag.get('href'))
        links_pdfs.append(a_tag.get('href'))

# generamos cada enlace
baselink = 'https://www.isciii.es'
for i in range(0, len(links_pdfs)):
    links_pdfs[i] = baselink + links_pdfs[i]

In [50]:
# descargamos un pdf para probar que funciona
